In [3]:
from logging import exception
import sqlalchemy as sa
import urllib
import pandas as pd
import numpy as np
parameters = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                    "SERVER=WIN-SERVIDOR-BD\DWGRUPOTOTAL;"
                                    "DATABASE=DW_Central;"
                                    "UID=sa;"
                                    "PWD=DWpln21")

In [60]:
df_original = pd.read_excel("y:/19-POWER BI/01-Total Bebidas/TACO GQ/Algoritmo Mayo.xlsb", sheet_name='Export', skipfooter=3)
df = df_original.copy(deep=True)
df = pd.DataFrame(df_original)
df['POC_ID'] = df['POC_ID'].astype('string')
# df.dtypes
# df.sample(10)

In [61]:
df['COD_CLIENTE'] = df.POC_ID.str.extract('(0{2,}\d+)')

In [63]:
df

,POC_ID,Ordenes Influenciadas,Ordenes inf. Upsell,Ordenes inf.Quick Order,Ordenes totales,COD_CLIENTE
0,4755100000004,0,0,0,10,00000004
1,4755100000006,2,1,2,2,00000006
2,4755100000010,1,0,1,2,00000010
3,4755100000019,1,0,1,3,00000019
4,4755100000023,3,1,3,3,00000023
...,...,...,...,...,...,...
112344,89955200021361,0,0,0,10,00021361
112345,89955200021362,0,0,0,8,00021362
112346,89955200021363,0,0,0,10,00021363
112347,89955200021364,2,2,0,11,00021364


In [ ]:
# df_m = df.loc[0:4,['POC_ID', 'Ordenes Influenciadas', 'Ordenes totales']] 
# df_m

In [203]:
df['Periodo'] = pd.to_datetime('1/6/2022')
df.convert_dtypes()
# df['POC_ID'].astype('Int64')
# df.dtypes

POC_ID                              int64
Ordenes Influenciadas               int64
Ordenes inf. Upsell                 int64
Ordenes inf.Quick Order             int64
Ordenes totales                     int64
Periodo                    datetime64[ns]
dtype: object

In [204]:
conditionlist = [
    (df['POC_ID']>=7042500000000) & (df['POC_ID']<7042600000000),
    (df['POC_ID']>=32866000000000) & (df['POC_ID']<32866100000000),
    (df['POC_ID']>=36911700000000) & (df['POC_ID']<36911800000000),
    (df['POC_ID']>=36984600000000) & (df['POC_ID']<36984700000000),
    (df['POC_ID']>=44562500000000) & (df['POC_ID']<44562600000000)]
choicelist = ['70425', '328660', '369117', '369846', '445625']
df['DISTRI'] = np.select(conditionlist, choicelist, default = 'OTRO DISTRI')
# df.sample

In [206]:
try: 
    df.drop('Ordenes inf. Upsell', axis=1, inplace=True)
except: 
    pass 
# df.groupby(['DISTRI', 'Periodo']).sum()
# df


In [207]:
# df.drop(columns='DISTRI'=='OTRO DISTRI',axis=0)
# df
df = df[df['DISTRI']!='OTRO DISTRI']
# df

In [214]:
def run():
    try:
        engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(parameters))
        print("Conect succesfull")
    except Exception:
        print("Errora in DataBase conection")
    try:    
        df.to_sql('Algoritmo', con = engine, if_exists= 'append', index=False)
        print("carga exitosa")
    except Exception:
        print("Error fatal irrecuperable")

run()


Errora in DataBase conection
Error fatal irrecuperable
